<a href="https://colab.research.google.com/github/ekinfergan/Thesis_Jupyter_Final/blob/main/src/models/mc_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Channel LSTM (Flexible Multi-Layer Selection)

In [ ]:
!git clone https://github.com/ekinfergan/Thesis_Jupyter_Final.git

Cloning into 'Thesis_Jupyter_Final'...
remote: Enumerating objects: 715, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 715 (delta 47), reused 133 (delta 45), pack-reused 580
Receiving objects: 100% (715/715), 136.60 MiB | 14.04 MiB/s, done.
Resolving deltas: 100% (365/365), done.
Filtering content: 100% (7/7), 127.99 MiB | 22.10 MiB/s, done.


In [1]:
%cd Thesis_Jupyter_Final
!git pull
%cd ..

/content/Thesis_Jupyter_Final
Already up to date.
/content


In [ ]:
!pip install --upgrade tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install git+https://github.com/scikit-optimize/scikit-optimize.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/scikit-optimize/scikit-optimize.git to /tmp/pip-req-build-2otwbmew
  Running command git clone --filter=blob:none --quiet https://github.com/scikit-optimize/scikit-optimize.git /tmp/pip-req-build-2otwbmew
  Resolved https://github.com/scikit-optimize/scikit-optimize.git to commit a2369ddbc332d16d8ff173b12404b03fea472492
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pyaml-23.5.9-py3-none-any.whl (17 kB)
  Created wheel for scikit-optimize: filename=scikit_optimize-0.9.0-py2.py3-none-any.whl size=100250 sha256=82182c375007bab602b44f2495b46be23bdba259741eefc87eeb46388755c1cf
  Stored in directory: /tmp/pip-ephem-wheel-cache-_baspxx_/wheels/2f/f0/ed/db529a96372d05bd34f6c3a2fa7c08ef7a8314315ac46e49d7
Successfully built scikit-optimize


In [2]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing
import pickle
from numpy import asarray

from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score, auc, roc_curve, RocCurveDisplay, confusion_matrix, classification_report
from sklearn.multiclass import OneVsRestClassifier
from itertools import cycle

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Embedding, concatenate, LSTM, Dense
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.legacy import Adam, SGD, RMSprop, Adagrad

import skopt
from skopt import gbrt_minimize, gp_minimize
from skopt.utils import use_named_args
from skopt.space import Real, Categorical, Integer
from tensorflow.keras import backend as K

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

  
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

  
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
  except RuntimeError as e:
    print(e)


import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
# just checkıng gpu ıs avaılable


try:
  resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
  tf.config.experimental_connect_to_cluster(resolver)
  tf.tpu.experimental.initialize_tpu_system(resolver)
  print("All devices: ", tf.config.list_logical_devices('TPU'))
  strategy = tf.distribute.experimental.TPUStrategy(resolver)
except ValueError:
  strategy = tf.distribute.get_strategy()

In [5]:
script_dir = os.path.dirname(os.path.abspath('mc_lstm.ipynb'))
data_path = os.path.join(script_dir, 'Thesis_Jupyter_Final/src/')
os.getcwd()
print(data_path)

input_folder_path = os.path.join(data_path, 'input')
processed_folder_path = os.path.join(data_path, 'input/processed')
results_folder_path =  os.path.join(data_path, "results/LSTM_results")

# Create the folder if it doesn't exist
if not os.path.exists(results_folder_path):
    os.makedirs(results_folder_path)

/content/Thesis_Jupyter_Final/src/


In [6]:
# global variables
# Define a dictionary to map sentiment values to category names
senti_labels = {1: 'Negative', 2: 'Neutral', 3: 'Positive'}
senti_categories = list(senti_labels.values())
NUM_of_CLASSES = 3

VOCAB_SIZE = 11395
MAX_SEQ_LEN = 449
EMBEDDING_DIM = 100
NUM_OUTPUT_CLASSES = 3

In [7]:
train = pd.read_csv(os.path.join(processed_folder_path, "train.csv"))
val = pd.read_csv(os.path.join(processed_folder_path, "val.csv"))
test = pd.read_csv(os.path.join(processed_folder_path, "test.csv"))

x_train = train['x']
y_train = train['y']
x_val = val['x']
y_val = val['y']
x_test = test['x']
y_test = test['y']
import pickle

# Load encoded sequences
with open(os.path.join(processed_folder_path, "x_train_encoded.pkl"), "rb") as f:
    x_train_encoded = pickle.load(f)
with open(os.path.join(processed_folder_path, "x_val_encoded.pkl"), "rb") as f:
    x_val_encoded = pickle.load(f)
with open(os.path.join(processed_folder_path, "x_test_encoded.pkl"), "rb") as f:
    x_test_encoded = pickle.load(f)
print(f"x_train_encoded:\n{x_train_encoded[:5]}\n")

# Load embedding vectors
with open(os.path.join(processed_folder_path, "embedding_matrix.pkl"), "rb") as f:
    w2v_embedding_vectors = pickle.load(f)
print(f"embedding vectors: {w2v_embedding_vectors[10][:5]}...\n")

# Load sentiment scores
with open(os.path.join(processed_folder_path, "x_train_scores_padded.pkl"), "rb") as f:
    x_train_scores = pickle.load(f)
with open(os.path.join(processed_folder_path, "x_val_scores_padded.pkl"), "rb") as f:
    x_val_scores = pickle.load(f)
with open(os.path.join(processed_folder_path, "x_test_scores_padded.pkl"), "rb") as f:
    x_test_scores = pickle.load(f)
print(f"x_train_scores:\n{x_train_scores[:5]}\n")

# Load encoded labels
with open(os.path.join(processed_folder_path, "y_train_encoded.pkl"), "rb") as f:
    y_train_encoded = pickle.load(f)
with open(os.path.join(processed_folder_path, "y_val_encoded.pkl"), "rb") as f:
    y_val_encoded = pickle.load(f)
with open(os.path.join(processed_folder_path, "y_test_encoded.pkl"), "rb") as f:
    y_test_encoded = pickle.load(f)
print(f"y_train_encoded:\n{y_train_encoded[:5]}\n")

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 1.19679999 -0.028458   -0.29611    ...  0.16068     0.16943
  -0.30098999]
 [ 0.66938001 -0.1402      0.080513   ...  0.1436      0.16368
  -0.23379   ]
 ...
 [ 0.54970002 -0.1337     -0.28555    ...  0.81467003  0.28928
  -0.072212  ]
 [-0.92633998  0.076536    0.13122    ... -0.89864999  0.1823
  -0.34810001]
 [ 1.42130005  0.16856    -0.56782001 ...  0.90786999  1.21930003
  -0.1842    ]]


## Evaluation Functions

In [8]:
def one_hot_encode(y):
    y_encoded = np.zeros((len(y), NUM_of_CLASSES))
    for i, label in enumerate(y):
        y_encoded[i, label - 1] = 1

    return y_encoded

def calculate_classification_report(y, y_pred, labels, target_names):
    report = classification_report(y, y_pred, labels=labels, target_names=target_names)
    print("Classification Report:\n", report)

def plot_confusion_matrix(y_true, y_pred, labels, res_path):
    cnf_mat = confusion_matrix(y_true, y_pred)
    mat_disp = ConfusionMatrixDisplay(confusion_matrix=cnf_mat, display_labels=labels)
    mat_disp = mat_disp.plot(cmap='Blues', xticks_rotation='vertical')
    plt.title(f'Confusion Matrix')
    plt.savefig(os.path.join(res_path, "confusion_matrix.png"))
    plt.close()

def plot_roc_curve(prob_test_vec, y_test, labels, res_path):
    fig, ax = plt.subplots(figsize=(10, 10))
    labels = labels
    colors = cycle(['limegreen', 'dodgerblue', 'red'])
    for senti, color in zip(range(NUM_of_CLASSES), colors):
        RocCurveDisplay.from_predictions(
            y_test[:, senti],
            prob_test_vec[:, senti],
            name=f"ROC curve for {labels[senti]}",
            color=color,
            ax=ax,
        )
    plt.savefig(os.path.join(res_path, "roc_curve.png"))
    plt.close()

def calculate_roc_auc_score(model, model_name, x_test, y_test, labels, res_path):
    prob_test_vec = model.predict(x_test)

    fpr, tpr, thresholds, auc_score = [], [], [], []
    for _ in range(NUM_of_CLASSES):
        fpr.append(0)
        tpr.append(0)
        thresholds.append(0)
        auc_score.append(0)

    for i in range(NUM_of_CLASSES):
        fpr[i], tpr[i], thresholds[i] = roc_curve(y_test[:, i], prob_test_vec[:, i])
        auc_score[i] = auc(fpr[i], tpr[i])

    averaged_auc_score = (sum(auc_score) / NUM_of_CLASSES)

    with open(os.path.join(res_path, f"{model_name}_results.txt"), "a") as f:
        f.write(f"AUC score: {auc_score}\n")
        f.write(f"Averaged AUC score: {averaged_auc_score:.2f}\n")

    plot_roc_curve(prob_test_vec, y_test, labels, res_path=res_path)


def plot_development(history):
    acc =  history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'b', label='Training Accuracy')
    plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()

    plt.figure()

    plt.plot(epochs, loss, 'b', label='Training Loss')
    plt.plot(epochs, val_loss, 'r', label='Validation Loss')
    plt.title('Training and validation Loss')
    plt.legend()
    plt.show()

    plt.savefig(os.path.join(results_folder_path, "development_plot.png"))
    plt.close()

def get_results(y_pred, y, x, score, history, model, model_name, params, res_path):
    if not os.path.exists(res_path):
        os.makedirs(res_path)

    # Convert to one hot vectors
    y_classes = np.argmax(y, axis=1) + 1
    y_pred_classes = np.argmax(y_pred, axis=1) + 1

    print(y_pred.shape)
    print(y_classes.shape)
    print(y_pred_classes.shape)

    print(f"Accuracy: {score[1]:.2%}")
    print(f"Loss: {score[0]:.2f}")

    with open(os.path.join(res_path, f"{model_name}_results.txt"), "w") as f:
        f.write(f"*{model_name}\n")
        f.write(f"Optimizer Params: {params}\n\n")

        f.write(f"Accuracy: {score[1]:.2f}%\n")
        f.write(f"Loss: {score[0]:.2f}\n")

        report = calculate_classification_report(y_classes, y_pred_classes, labels=np.unique(y_classes), target_names=senti_categories)
        if report is not None:
            f.write("Classification Report:\n")
            f.write(report)
        else:
            print("Failed to generate classification report")
        f.write("\n")

        plot_confusion_matrix(y_classes, y_pred_classes, senti_categories, res_path)
        plot_development(history)
        #calculate_roc_auc_score(model, model_name, x, y, senti_categories, res_path)

# JUST TO CHECK IF A SIMPLE MODEL IS RUNABLE AT ALL

In [10]:
model = Sequential()
# input layer is sequence of integers (words)
model.add(Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_SEQ_LEN, name="embedding_layer")) # part of input layer as it transforms integers into dense vectors, input shape = (None, MAX_SEQ_LEN)
model.add(LSTM(64, name='hidden_layer')) # hidden layer
model.add(Dense(NUM_OUTPUT_CLASSES, activation='softmax', name="output_layer"))
model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=["accuracy"])
model.summary()

basic_history = model.fit(x_train_encoded, y_train_encoded, validation_data=(x_val_encoded, y_val_encoded), batch_size=16, epochs=3)

y_pred = model.predict(x_test_encoded)
score = model.evaluate(x_test_encoded, y_test_encoded)

get_results(y_pred, y_test_encoded, x_test_encoded, score, history, model, "LSTM", model.optimizer.get_config(), results_folder_path)

del model

K.clear_session()
tf.compat.v1.reset_default_graph()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_layer (Embedding)  (None, 430, 100)         1135400   
                                                                 
 hidden_layer (LSTM)         (None, 64)                42240     
                                                                 
 output_layer (Dense)        (None, 3)                 195       
                                                                 
Total params: 1,177,835
Trainable params: 1,177,835
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
  37/2563 [..............................] - ETA: 7:57 - loss: 1.0700 - accuracy: 0.4595

KeyboardInterrupt: ignored

## Hypterparameter Tuning

In [28]:
batch_size= 16
epochs=30

num_lstm_layersA = Integer(low=1, high=3, name='num_lstm_layersA')
num_lstm_layersB = Integer(low=1, high=3, name='num_lstm_layersB')
num_lstm_unitsA = Categorical([32, 64, 128], name='num_lstm_unitsA')
num_lstm_unitsB = Categorical([32, 64, 128], name='num_lstm_unitsB')
learning_rate = Categorical([1e-2, 1e-3], name='learning_rate')
#adam_decay = Categorical([1e-6, 1e-4, 1e-2], name="adam_decay")

search_space = [
            num_lstm_layersA,
            num_lstm_layersB,
            num_lstm_unitsA,
            num_lstm_unitsB,
            learning_rate
            ]

# Specify one or more initial points for the search of optimal parameter
default_params = [2,
                  2,
                  32,
                  32,
                  1e-2
                  ]

In [23]:
def define_multi_channel_lstm_model(num_lstm_layersA, num_lstm_layersB, num_lstm_unitsA, num_lstm_unitsB, learning_rate):
    # Vocabulary-based embedding layer
    inputsA = Input(shape=(MAX_SEQ_LEN,), name="input regular embeddings")
    # Word2Vec embedding layer
    inputsB = Input(shape=(MAX_SEQ_LEN,), name="input word2vec embeddings")

    # Define an embedding layer for each input
    embeddingsA = Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_SEQ_LEN, name="embeddingsA")(inputsA)
    embeddingsB = Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_SEQ_LEN, weights=[w2v_embedding_vectors], trainable=False, name="embeddingsB")(inputsB)

    # Pass both embeddings through their own LSTM layers
    lstm_layersA = embeddingsA
    for i in range(num_lstm_layersA):
        nameA = 'layer_lstmA_{0}'.format(i+1)
        if i < num_lstm_layersA-1:
            lstm_layersA = LSTM(num_lstm_unitsA, return_sequences=True, name=nameA)(lstm_layersA)
        else:
            lstm_layersA = LSTM(num_lstm_unitsA, return_sequences=False, name=nameA)(lstm_layersA)

    lstm_layersB = embeddingsB
    for i in range(num_lstm_layersB):
        nameA = 'layer_lstmB_{0}'.format(i+1)
        if i < num_lstm_layersB-1:
            lstm_layersB = LSTM(num_lstm_unitsB, return_sequences=True, name=nameA,)(lstm_layersB)
        else:
            lstm_layersB = LSTM(num_lstm_unitsB, return_sequences=False, name=nameA)(lstm_layersB)


    # Concatenate the two inputs
    merged = concatenate([lstm_layersA, lstm_layersB])

    # Additional Dense layer for dimensionality reduction
    dense_layer = Dense(16, activation='relu', name="dense_layer")(merged)

    # Dense layer for the merged inputs & output Layer
    outputs = Dense(NUM_OUTPUT_CLASSES, activation='softmax', name="output")(dense_layer)

    # Create the model
    model = Model(inputs=[inputsA, inputsB], outputs=outputs)

    # Compile the model
    rmsprop = RMSprop(learning_rate=learning_rate) # default is 0.001
    model.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics=['accuracy']) # default learning rate = 0.001
    print(model.summary())

    return model

In [26]:
@use_named_args(dimensions=search_space)
def multi_objective_function(num_lstm_layersA, num_lstm_layersB, num_lstm_unitsA, num_lstm_unitsB, learning_rate):

    model = define_multi_channel_lstm_model(num_lstm_layersA=num_lstm_layersA,
                                            num_lstm_layersB=num_lstm_layersB,
                                            num_lstm_unitsA=num_lstm_unitsA,
                                            num_lstm_unitsB=num_lstm_unitsB,
                                            learning_rate=learning_rate
                                            )

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    history = model.fit([x_train_encoded, x_train_encoded],
                        y_train_encoded,
                        validation_data=([x_val_encoded, x_val_encoded], y_val_encoded),
                        epochs=epochs, # TODO
                        batch_size=batch_size,
                        callbacks=[early_stopping],
                        verbose=1
                        )
    #return the validation accuracy for the last epoch.
    accuracy = history.history['val_accuracy'][-1]
    loss = history.history['val_loss'][-1]

    # Print the classification accuracy.
    print(f"Accuracy: {accuracy:.2%}")
    print(f"Loss: {loss:.2}\n")


    # Delete the Keras model with these hyper-parameters from memory.
    del model

    K.clear_session()
    tf.compat.v1.reset_default_graph()

    # Tthe optimizer aims for the lowest score, so we return our negative accuracy
    return -accuracy

## Gradient Boosted Model

In [29]:
gbrt_result = gbrt_minimize(func=multi_objective_function,
                            dimensions=search_space,
                            n_calls=12,
                            n_jobs=-1,
                            x0=default_params)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input regular embeddings (Inpu  [(None, 430)]       0           []                               
 tLayer)                                                                                          
                                                                                                  
 input word2vec embeddings (Inp  [(None, 430)]       0           []                               
 utLayer)                                                                                         
                                                                                                  
 embeddingsA (Embedding)        (None, 430, 100)     1135400     ['input regular embeddings[0][0]'
                                                                 ]                          

KeyboardInterrupt: ignored

In [ ]:
# TODO data frame summarizing parameter search
gbrt_best_params = {param.name: value for param, value in zip(gbrt_result.space, gbrt_result.x)}
print("Best Hyperparameters:", gbrt_best_params)

In [ ]:
model = define_multi_channel_lstm_model(gbrt_best_params['num_lstm_layersA'],
                                        gbrt_best_params['num_lstm_layersB'],
                                        gbrt_best_params['num_lstm_unitsA'],
                                        gbrt_best_params['num_lstm_unitsB'],
                                        gbrt_best_params['learning_rate']
                                        )

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True) # TODO: should I, again?
history = model.fit([x_train_encoded, x_train_encoded],
                    y_train_encoded,
                    validation_data=([x_val_encoded, x_val_encoded], y_val_encoded),
                    epochs=epochs, # TODO
                    batch_size=batch_size,
                    callbacks=[early_stopping],
                    verbose=2
                    )
plot_development(history)

In [ ]:
model.evaluate([x_train_encoded, x_train_encoded], y_train_encoded)
evaluate_model(model, "Train multi-LSTM", x_train_encoded, y_train_encoded, only_metrics=True)

model.evaluate([x_val_encoded, x_val_encoded], y_val_encoded, verbose=0)
evaluate_model(model, "Val multi-LSTM", x_val_encoded, y_val_encoded, only_metrics=True)

model.evaluate([x_test_encoded, x_test_encoded], y_test_encoded, verbose=0)
evaluate_model(model, "Test multi-LSTM", x_test_encoded, y_test_encoded, y_test, only_metrics=False)
senti_labels = ['negative', 'neutral', 'positive'] # TODO
#calculate_OvR_roc_auc_score(model, x_train, y_train, x_test, y_test, senti_labels)

## Gaussian Process Model

In [ ]:
gp_result = gp_minimize(func=multi_objective_function,
                            dimensions=search_space,
                            n_calls=12,
                            noise= 0.01,
                            n_jobs=-1,
                            kappa = 5,
                            x0=default_params)

In [ ]:
# TODO data frame summarizing parameter search
gp_best_params = {param.name: value for param, value in zip(gp_result.space, gp_result.x)}
print("Best Hyperparameters:", gp_best_params)

In [ ]:
model = define_multi_channel_lstm_model(gp_best_params['num_lstm_layersA'],
                                        gp_best_params['num_lstm_layersB'],
                                        gp_best_params['num_lstm_unitsA'],
                                        gp_best_params['num_lstm_unitsB'],
                                        gp_best_params['learning_rate'],
                                        gp_best_params['adam_decay']
                                        )

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True) # TODO: should I, again?
history = model.fit([x_train_encoded, x_train_encoded],
                    y_train_encoded,
                    validation_data=([x_val_encoded, x_val_encoded], y_val_encoded),
                    epochs=epochs, # TODO
                    batch_size=batch_size,
                    callbacks=[early_stopping],
                    verbose=2
                    )
plot_development(history)

In [ ]:
# TODO: Get reults may need adjusting
y_pred = model.predict([x_train_encoded, x_train_encoded])
score = model.evaluate([x_train_encoded, x_train_encoded], y_test_encoded)
get_results(y_pred, y_train_encoded, x_train_encoded, score, history, model, "Train multi-LSTM", model.optimizer.get_config(), results_folder_path)

y_pred = model.predict([x_val_encoded, x_val_encoded])
score = model.evaluate([x_val_encoded, x_val_encoded], y_val_encoded)
get_results(y_pred, y_test_encoded, x_test_encoded, score, history, model, "Val multi-LSTM", model.optimizer.get_config(), results_folder_path)

y_pred = model.predict([x_test_encoded, x_test_encoded])
score = model.evaluate([x_test_encoded, x_test_encoded], y_test_encoded)
get_results(y_pred, y_test_encoded, x_test_encoded, score, history, model, "Test multi-LSTM", model.optimizer.get_config(), results_folder_path)